## Classification

In [14]:
from tqdm import tqdm

import medmnist
from medmnist import DermaMNIST
from medmnist import INFO, Evaluator

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

print(f"MedMnist v{medmnist.__version__}")

MedMnist v3.0.1


### 1. Load, Preprocss and Normalize MedMnist data

In [15]:
DataClass = getattr(medmnist, INFO['dermamnist']['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=True)
test_dataset = DataClass(split='test', transform=data_transform, download=True)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*128, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*128, shuffle=False)

Using downloaded and verified file: /Users/xaver/.medmnist/dermamnist.npz
Using downloaded and verified file: /Users/xaver/.medmnist/dermamnist.npz


### 2. Define a Convolutional Neural Network
Takes 3 channels, as it's colored.

In [29]:
# copied from lecture

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)

    def forward(self, x):
        x = self.conv1(x)
#         print(x.shape)
        x = self.pool(F.relu(x))
#         print(x.shape)
        x = self.conv2(x)
#         print(x.shape)
        x = self.pool(F.relu(x))
#         print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         print(x.shape)
        return x


model = Net()

### 3. Define a Loss function and optimizer

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

### 4. Train the network

In [31]:
for epoch in range(3):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    
    model.train()
    for inputs, targets in tqdm(train_loader):
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)
        

        targets = targets.squeeze().long()
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

print('Finished Training')

100%|██████████| 55/55 [00:01<00:00, 47.53it/s]

Finished Training


### 5. Test the network

In [36]:
# evaluation

def test(split):
    model.eval()
    y_true = torch.tensor([])
    y_score = torch.tensor([])
    
    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)

            targets = targets.squeeze().long()
            outputs = outputs.softmax(dim=-1)
            targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.numpy()
        y_score = y_score.detach().numpy()
        evaluator = Evaluator('dermamnist', split)
        metrics = evaluator.evaluate(y_score)
    
        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))

        
print('==> Evaluating ...')
test('train')
test('test')

==> Evaluating ...
train  auc: 0.469  acc:0.369
test  auc: 0.440  acc:0.356
